In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Initial parameters
P_0 = 300000  # Thermal power (kW)
Delta_T = 500  # Temperature difference between fuel and coolant (°C)
Tin_0 = 400    # Initial inlet temperature (°C)
M_f = 2132.0   # Fuel mass (kg)
C_f = 0.376     # Specific heat of the fuel (kJ/(kg*K))
M_c = 5429    # Coolant mass (kg)
C_c = 0.146     # Specific heat of the coolant (kJ/(kg*K))
G = 25757      # Fluid flow rate (kg/s)
L = 0.8066e-5  # Mean generation time (s)
b = 319  # Total delayed neutron fraction

# Delayed neutron data
li = np.array([0.0125, 0.0292, 0.0895, 0.2575, 0.6037, 2.6688])  # Decay constants (s^-1)
bi = np.array([6.142, 71.40, 34.86,114.1, 69.92, 22.68])  # Fraction of delayed neutrons

# Temperature feedback coefficients (pcm/K)
alpha_D = -0.17  # Doppler
alpha_C = -1.995 # Coolant density
alpha_A = -0.2374  # Axial expansion
alpha_R = -0.7144  # Radial expansion
# Feedback coefficients
alpha_f = alpha_D + alpha_A # Fuel feedback coefficient (pcm/°C)
alpha_c = alpha_C + alpha_R # Coolant feedback coefficient (pcm/°C)
alpha_h = 10/(10**5)   # Control rods feedback coefficient (pcm/cm)

# Calculated parameters
l = (np.sum(bi / li) / b)**(-1)  # Effective neutron lifetime
C_0 = (P_0 / L) * (bi / li)  # Delayed neutron precursors
K = P_0 / Delta_T  # Total heat transfer coeff. (kW/°C)
tau_f = M_f * C_f / K  # Fuel time constant (s)
tau_c = M_c * C_c / K  # Coolant time constant (s)
tau_0 = M_c / G  # Transport time constant (s)

# Initial conditions
rho_0=0
TH_0 = [Tin_0 + P_0 / (2 * G * C_c) + Delta_T, Tin_0 + P_0 / (2 * G * C_c)]
NEU_0 = np.append(P_0, C_0)
PWR_0 = np.append(NEU_0, TH_0)

# Time points
t_start = 0
dt=0.1 #s
t_end = 50  # Simulation time (s)
t_eval = np.arange(t_start, t_end, step=dt)  # time step = dt from 0 to T_final seconds

# System model
def Dyn6_model(t, state):
    P, C1, C2, C3, C4, C5, C6, Tf, Tc = state
    dPdt = (alpha_h * U[0] + alpha_f * (Tf - TH_0[0]) + alpha_c * (Tc - TH_0[1]) - b) * P / L + li[0] * C1 + li[1] * C2 + li[2] * C3 + li[3] * C4 + li[4] * C5 + li[5] * C6
    dCdt = [(bi[i] / L) * P - li[i] * state[i + 1] for i in range(6)]
    dTfdt = P / (M_f * C_f) - K * (Tf - Tc) / (M_f * C_f)
    dTcdt = K * (Tf - Tc) / (M_c * C_c) - 2 * G * (Tc - U[1]) / M_c
    return [dPdt] + dCdt + [dTfdt, dTcdt]
# Inputs
rho_0 = 0
dh = 10  # Height change
U = [dh, Tin_0]  # Input vector


# Solve the system of ODEs
sol = solve_ivp(Dyn6_model,(t_start, t_end), PWR_0, t_eval=t_eval, method='RK45', rtol=1e-6, atol=1e-8)

# Plot results
labels_extended = ['Power (GW)', 'Precursor C1', 'Precursor C2', 'Precursor C3', 'Precursor C4', 'Precursor C5',
                   'Precursor C6', 'Fuel temperature (°C)', 'Coolant temperature (°C)', 'Outlet temperature (°C)',
                   'Reactivity (pcm)']
responses_extended = list(sol.y[:9]) + [2 * sol.y[8] - U[1],
    (alpha_h * U[0] + alpha_f * (sol.y[7] - TH_0[0]) + alpha_c * (sol.y[8] - TH_0[1])) * 1e5]

for i, response in enumerate(responses_extended):
    plt.figure()
    plt.plot(t_eval, response, label=labels_extended[i])
    plt.xlabel('Time (s)')
    plt.ylabel(labels_extended[i])
    plt.legend(loc='best')
    plt.grid()
    plt.show()

In [ ]:
labels_variations = ['Power variation (MW)', 'Precursor C1 variation', 'Precursor C2 variation',
                     'Precursor C3 variation', 'Precursor C4 variation', 'Precursor C5 variation',
                     'Precursor C6 variation', 'Fuel temperature variation (°C)',
                     'Coolant temperature variation (°C)', 'Outlet temperature variation (°C)',
                     'Reactivity variation (pcm)']
responses_variations = [(sol.y[i] - PWR_0[i]) for i in range(9)] + [
    2 * sol.y[8] - U[1] - (2 * PWR_0[8] - Tin_0),
    (alpha_h * U[0] + alpha_f * (sol.y[7] - TH_0[0]) + alpha_c * (sol.y[8] - TH_0[1])) * 1e5
]

for i, response in enumerate(responses_variations):
    plt.figure()
    plt.plot(t_eval, response, label=labels_variations[i],color='orange')
    plt.xlabel('Time (s)')
    plt.ylabel(labels_variations[i])
    plt.legend(loc='best')
    plt.grid()
    plt.show()